In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import MarkerStyle as markerstyle
import math
from scipy.interpolate import griddata
import pandas
from scipy import constants
import matplotlib.patches as patches
import matplotlib.cm as cm

In [ ]:
"""
------- Defining Constants ------------
"""
k = constants.value('Boltzmann constant')
Ava_no = constants.value('Avogadro constant')
temp = 310
kbt = (k*temp*Ava_no)/1000     #kJ/mol

In [ ]:
#Energies_File
#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

rmsdbd_file = "../3D_Martini2/RMSF/CRYSTAL/AllBound_RMSD_chainA_CRYSTAL.dat"
rmsd_unb_file = "../3D_Martini2/RMSF/CRYSTAL/AllUnbound_RMSD_chainA_CRYSTAL.dat"

rmsd_bd_data3D = pandas.read_csv(rmsdbd_file,comment='#',sep='\s+',dtype=np.float64)
rmsd_unb_data3D = pandas.read_csv(rmsd_unb_file,comment='#',sep='\s+',dtype=np.float64)


# ener_col = ['Timestep','Bond','G96','Imp','LJ','Col','Pot','Volume','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W','Col-W-W','LJ-W-W']
ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)



# #Merge two dataframes
master_data3D = dist_data.join(ener_data.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')
# master_data3D = dist_data

In [ ]:
bound_mask = (master_data3D['d1_mda'] < 13.0) & (master_data3D['d2_mda'] < 13.0)
# # bound_mask = (master_data['RMSD-BtoA'] < 2.0)
# unbound_mask = ((master_data3D['d1_mda'] > 13.0) | (master_data3D['d2_mda'] > 13.0))

# bound_mask = (master_data3D['d1'] < 13.0) & (master_data3D['d2'] < 13.0)
# unbound_mask = ((master_data3D['d1'] > 13.0) | (master_data3D['d2'] > 13.0))


bd_data3D = master_data3D[bound_mask]
unb_data3D = master_data3D[~bound_mask]

In [ ]:
#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

rmsdbd_file = "../2D_Martini2/AllBound_RMSD_chainA_CRYSTAL.dat"
rmsd_unb_file = "../2D_Martini2/AllUnBound_RMSD_chainA_CRYSTAL.dat"

rmsd_bd_data2D = pandas.read_csv(rmsdbd_file,comment='#',sep='\s+',dtype=np.float64)
rmsd_unb_data2D = pandas.read_csv(rmsd_unb_file,comment='#',sep='\s+',dtype=np.float64)

# ener_col = ['Timestep','Bond','G96','ImPDih','LJ','Col','Pot','Col-Mem-Mem','LJ-Mem-Mem','Col-Mem-W','LJ-Mem-W','Col-Pro-Mem','LJ-Pro-Mem','Col-W-W','LJ-W-W','Col-Pro-W','LJ-Pro-W','Col-Pro-Pro','LJ-Pro-Pro']

ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)

mask1 = dist_data['Timestep']%1000 == 0
dist_data02 = dist_data[mask1]
dist_data02.reset_index(drop=True,inplace=True)
mask1 = ener_data['Timestep']<= 220000000
ener_data02 = ener_data[mask1]
ener_data02.reset_index(drop=True,inplace=True)

# #Merge two dataframes
master_data2D = dist_data02.join(ener_data02.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

In [ ]:
bound_mask = (master_data2D['d1_mda'] < 13.0) & (master_data2D['d2_mda'] < 13.0)
# bound_mask = (master_data['RMSD-BtoA'] < 2.0)
# unbound_mask = ((master_data2D['d1_mda'] > 13.0) | (master_data2D['d2_mda'] > 13.0))

bd_data2D = master_data2D[bound_mask]
unb_data2D = master_data2D[~bound_mask]

In [ ]:
"""
Pseudo MEmbrane Values
"""
rmsdbd_file = "../PseudoMart2/AllBound_RMSD_chainA_CRYSTAL.dat"
rmsd_unb_file = "../PseudoMart2/AllUnBound_RMSD_chainA_CRYSTAL.dat"

rmsd_bd_data_pseudo = pandas.read_csv(rmsdbd_file,comment='#',sep='\s+',dtype=np.float64)
rmsd_unb_data = pandas.read_csv(rmsd_unb_file,comment='#',sep='\s+',dtype=np.float64)

In [ ]:
f_size=22
t_size=20
lw=1.6
leg_size=18
fig_size=(6,7)

In [ ]:
fig,[ax,ax1] = plt.subplots(2,1,figsize=fig_size,sharex=True)
nbins=60
# hist1 = ax.hist(bd_data3D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='3D',color='navy')
hist1 = ax.hist(rmsd_bd_data3D['RMSD'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')

# hist2 = ax.hist(bd_data2D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')
hist2 = ax.hist(rmsd_bd_data2D['RMSD'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')


hist3 = ax.hist(rmsd_bd_data_pseudo['RMSD'],bins=nbins,alpha=0.5,density=True,label='2D-Ps',color='teal')

bin_center1 = [(hist1[1][i]+hist1[1][i+1])/2 for i in range(len(hist1[1])-1)]
bin_center2 = [(hist2[1][i]+hist2[1][i+1])/2 for i in range(len(hist2[1])-1)]
bin_center3 = [(hist3[1][i]+hist3[1][i+1])/2 for i in range(len(hist3[1])-1)]

ax.plot(bin_center1,hist1[0],color='navy',lw=lw,alpha=0.8)
ax.plot(bin_center2,hist2[0],color='firebrick',lw=lw,alpha=0.8)
ax.plot(bin_center3,hist3[0],color='teal',lw=lw,alpha=0.8)

ax.legend(fontsize=leg_size, loc='upper left',frameon=False)
# ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size)
ax.tick_params(labelsize=t_size)
ax.set_ylabel(r'$PDF$',fontsize=f_size)
# ax.set_xticks(np.arange(0,1.5,0.2))
# ax.set_xlim(0,2.0)
ax.set_ylim(0,6)

#Unbound
hist1 = ax1.hist(unb_data3D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='3D',color='navy')
# hist2 = ax1.hist(unb_data2D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')
hist2 = ax1.hist(rmsd_unb_data['RMSD'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')
# hist3 = ax.hist(rmsd_bd_data_pseudo['RMSD-A'],bins=nbins,alpha=0.5,density=True,label='2D-Pseudo',color='teal')

bin_center1 = [(hist1[1][i]+hist1[1][i+1])/2 for i in range(len(hist1[1])-1)]
bin_center2 = [(hist2[1][i]+hist2[1][i+1])/2 for i in range(len(hist2[1])-1)]
# bin_center3 = [(hist3[1][i]+hist3[1][i+1])/2 for i in range(len(hist3[1])-1)]

ax1.plot(bin_center1,hist1[0],color='navy',lw=lw,alpha=0.8)
ax1.plot(bin_center2,hist2[0],color='firebrick',lw=lw,alpha=0.8)
# ax.plot(bin_center3,hist3[0],color='teal',lw=lw)

# ax1.legend(fontsize=leg_size)
ax1.set_xlabel(r'$RMSD |_{cry}$ (nm)',fontsize=f_size)
ax1.tick_params(labelsize=t_size)
ax1.set_ylabel(r'$PDF$',fontsize=f_size)
# ax.set_xticks(np.arange(0.2,1,0.2))
ax1.set_xticks(np.arange(0,2.1,0.4))
ax1.set_xlim(0,1.2)
ax1.set_ylim(0,6)
fig.tight_layout()
# plt.savefig('RMSD_CRY_Mart2.png',dpi=300,format='png')
# plt.savefig('RMSD_CRY_Mart2.svg',dpi=300,format='svg',transparent=True)


# fig.tight_layout()
plt.savefig('RMSD_Bound_CRY_Mart2.png',dpi=300,format='png',transparent=True)
plt.savefig('RMSD_Bound_CRY_Mart2.svg',dpi=300,format='svg',transparent=True)

In [ ]:
### CDF plot
pdf_3D_bd = np.histogram(bd_data3D['RMSD-A'],bins=nbins,density=True)
pdf_3D_unbd = np.histogram(unb_data3D['RMSD-A'],bins=nbins,density=True)

# pdf_2D_bd = np.histogram(bd_data2D['RMSD-A'],bins=nbins,density=True)
# pdf_2D_unbd = np.histogram(unb_data2D['RMSD-A'],bins=nbins,density=True)

pdf_2D_bd = np.histogram(rmsd_bd_data['RMSD'],bins=nbins,density=True)
pdf_2D_unbd = np.histogram(rmsd_unb_data['RMSD'],bins=nbins,density=True)


pdf_pseudo_bd = np.histogram(rmsd_bd_data_pseudo['RMSD'],bins=nbins,density=True)
# pdf_pseudo_unbd = np.histogram(rmsd_unb_data_pseudo['RMSD'],bins=nbins,density=True)

cdf_3D_bd = np.cumsum(pdf_3D_bd[0])*(pdf_3D_bd[1][1]-pdf_3D_bd[1][0])
cdf_3D_unbd = np.cumsum(pdf_3D_unbd[0])*(pdf_3D_unbd[1][1]-pdf_3D_unbd[1][0])

cdf_2D_bd = np.cumsum(pdf_2D_bd[0])*(pdf_2D_bd[1][1]-pdf_2D_bd[1][0])
cdf_2D_unbd = np.cumsum(pdf_2D_unbd[0])*(pdf_2D_unbd[1][1]-pdf_2D_unbd[1][0])

cdf_pseudo_bd = np.cumsum(pdf_pseudo_bd[0])*(pdf_pseudo_bd[1][1]-pdf_pseudo_bd[1][0])
# cdf_pseudo_unbd = np.cumsum(pdf_pseudo_unbd[0])*(pdf_pseudo_unbd[1][1]-pdf_pseudo_unbd[1][0])

fig,ax = plt.subplots(figsize=(4,3))


ax.plot(pdf_3D_bd[1][:-1],cdf_3D_bd,color='navy',lw=lw+2,label='3D')
ax.plot(pdf_2D_bd[1][:-1],cdf_2D_bd,color='firebrick',lw=lw+2,label='2D')
ax.plot(pdf_pseudo_bd[1][:-1],cdf_pseudo_bd,color='teal',lw=lw+2,label='2D-Pseudo')

# ax.legend(fontsize=legsize)
ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size+1)
ax.tick_params(labelsize=t_size+3)
ax.set_ylabel(r'$CDF$',fontsize=f_size+1)
ax.set_xticks([0.0,0.6,1.2])
ax.set_xlim(0,1.2)

fig.tight_layout()
plt.savefig("RMSD_CDF_Bound_CryMart2.svg",dpi=600,format="svg",transparent=True)
plt.show()



"""
CDF-Unbound
"""
fig,ax = plt.subplots(figsize=(4,3))


ax.plot(pdf_3D_unbd[1][:-1],cdf_3D_unbd,color='navy',lw=lw+2,label='3D')
ax.plot(pdf_2D_unbd[1][:-1],cdf_2D_unbd,color='firebrick',lw=lw+2,label='2D')
# ax.plot(pdf_pseudo_unbd[1][:-1],cdf_pseudo_unbd,color='teal',lw=lw+2,label='2D-Pseudo')

# ax.legend(fontsize=legsize)
ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size+3)
ax.tick_params(labelsize=t_size+3)
ax.set_ylabel(r'$CDF$',fontsize=f_size+3)
ax.set_xticks([0.0,0.6,1.2])
ax.set_xlim(0,1.2)

fig.tight_layout()
plt.savefig("RMSD_CDF_Unbound_CryMart2.svg",dpi=600,format="svg",transparent=True)
plt.show()

In [ ]:
"""
------- Defining Constants ------------
"""
k = constants.value('Boltzmann constant')
Ava_no = constants.value('Avogadro constant')
temp = 310
kbt = (k*temp*Ava_no)

# RMSD distribution w.r.t Avg Structure


In [ ]:
#Energies_File
#Small Box - No SALT
# ener_file = "../3D_Martini2/All_Energies_200us"
# dist_file = "../3D_Martini2/Final_data_rmsd_132us.txt"

ener_file = "../3D_Martini2/Reweigthed_Data_FINAL_100NaCl"


# ener_col = ['Timestep','Bond','G96','Imp','LJ','Col','Pot','Volume','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W','Col-W-W','LJ-W-W']
ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
rmsd_bd_data3D = pandas.read_csv("../3D_Martini2/RMSD_wrt_BoundAveraged_Struct_chainAB.pdb_.dat",sep="\t",comment='#',dtype=np.float64)
rmsd_unb_data3D = pandas.read_csv("../3D_Martini2/RMSD_wrt_UnboundAveraged_Struct_chainAB.pdb_.dat",sep="\t",comment='#')


#Remove duplicates 
rmsd_bd_data3D = rmsd_bd_data3D.drop_duplicates(subset=['Time'],ignore_index=True)
rmsd_und_data3D = rmsd_unb_data3D.drop_duplicates(subset=['Time'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)




In [ ]:
bd_timesteps = rmsd_bd_data3D['Time']
#Extract these timesteps from the energy file
bd_data = ener_data[ener_data['Timestep'].isin(bd_timesteps)]


bd_data.reset_index(drop=True,inplace=True)


unb_timesteps = rmsd_unb_data3D['Time']
#Extract these timesteps from the energy file
unb_data = ener_data[ener_data['Timestep'].isin(unb_timesteps)]
unb_data.reset_index(drop=True,inplace=True)

print(len(bd_data), len(rmsd_bd_data3D))
print(len(unb_data), len(rmsd_unb_data3D))

#Only join the rbias column from the energy file
merged_bd_data3D = rmsd_bd_data3D.join(bd_data.set_index('Timestep'),on='Time',how='inner',lsuffix='_rmsd',rsuffix='_ener')
merged_unb_data3D = rmsd_unb_data3D.join(unb_data.set_index('Timestep'),on='Time',how='inner',lsuffix='_rmsd',rsuffix='_ener')



In [ ]:
#Energies_File
ener_file = "../2D_Martini2/Reweigthed_Data_FINAL"

ener_data2D = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
rmsd_bd_data2D = pandas.read_csv("../2D_Martini2/RMSD_wrt_BoundAveraged_Struct_chainAB.pdb_.dat",sep="\t",comment='#')
rmsd_unb_data2D = pandas.read_csv("../2D_Martini2/RMSD_wrt_UnboundAveraged_Struct_chainAB.pdb_.dat",sep="\t",comment='#')


#Remove duplicates 
rmsd_bd_data2D = rmsd_bd_data2D.drop_duplicates(subset=['Time'],ignore_index=True)
rmsd_und_data2D = rmsd_unb_data2D.drop_duplicates(subset=['Time'],ignore_index=True)
ener_data2D = ener_data2D.drop_duplicates(subset=['Timestep'],ignore_index=True)



In [ ]:
bd_timesteps = rmsd_bd_data2D['Time']
#Extract these timesteps from the energy file
bd_data = ener_data2D[ener_data2D['Timestep'].isin(bd_timesteps)]

bd_data.reset_index(drop=True,inplace=True)

unb_timesteps = rmsd_unb_data2D['Time']
#Extract these timesteps from the energy file
unb_data = ener_data2D[ener_data2D['Timestep'].isin(unb_timesteps)]
unb_data.reset_index(drop=True,inplace=True)

print(len(bd_data), len(rmsd_bd_data2D))
print(len(unb_data), len(rmsd_unb_data2D))

#Only join the rbias column from the energy file
merged_bd_data2D = rmsd_bd_data2D.join(bd_data.set_index('Timestep'),on='Time',how='inner',lsuffix='_rmsd',rsuffix='_ener')
merged_unb_data2D = rmsd_unb_data2D.join(unb_data.set_index('Timestep'),on='Time',how='inner',lsuffix='_rmsd',rsuffix='_ener')


### Plot the RMSD distributions w.r.t Avg structure

In [ ]:
fig,[ax,ax1] = plt.subplots(2,1,figsize=fig_size,sharex=True)
nbins=60
hist1 = ax.hist(merged_bd_data3D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='3D',color='navy')
hist2 = ax.hist(merged_bd_data2D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')
# hist3 = ax.hist(rmsd_bd_data_pseudo['RMSD-A'],bins=nbins,alpha=0.5,density=True,label='2D-Pseudo',color='teal')

bin_center1 = [(hist1[1][i]+hist1[1][i+1])/2 for i in range(len(hist1[1])-1)]
bin_center2 = [(hist2[1][i]+hist2[1][i+1])/2 for i in range(len(hist2[1])-1)]
# bin_center3 = [(hist3[1][i]+hist3[1][i+1])/2 for i in range(len(hist3[1])-1)]

ax.plot(bin_center1,hist1[0],color='navy',lw=lw,alpha=0.8)
ax.plot(bin_center2,hist2[0],color='firebrick',lw=lw,alpha=0.8)
# ax.plot(bin_center3,hist3[0],color='teal',lw=lw)

# ax.legend(fontsize=leg_size)
# ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size)
ax.tick_params(labelsize=t_size)
ax.set_ylabel(r'$PDF$',fontsize=f_size)
ax.set_xticks(np.arange(0.0,1.2,0.2))
ax.set_xlim(0,0.9)
ax.set_ylim(0,7)

hist1 = ax1.hist(merged_unb_data3D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='3D',color='navy')
hist2 = ax1.hist(merged_unb_data2D['RMSD-A'],bins=nbins,density=True,alpha=0.4,label='2D',color='firebrick')
# hist3 = ax.hist(rmsd_bd_data_pseudo['RMSD-A'],bins=nbins,alpha=0.5,density=True,label='2D-Pseudo',color='teal')

bin_center1 = [(hist1[1][i]+hist1[1][i+1])/2 for i in range(len(hist1[1])-1)]
bin_center2 = [(hist2[1][i]+hist2[1][i+1])/2 for i in range(len(hist2[1])-1)]
# bin_center3 = [(hist3[1][i]+hist3[1][i+1])/2 for i in range(len(hist3[1])-1)]

ax1.plot(bin_center1,hist1[0],color='navy',lw=lw,alpha=0.8)
ax1.plot(bin_center2,hist2[0],color='firebrick',lw=lw,alpha=0.8)
# ax.plot(bin_center3,hist3[0],color='teal',lw=lw)

# ax1.legend(fontsize=leg_size)
ax1.set_xlabel(r'$RMSD |_{avg}$ (nm)',fontsize=f_size)
ax1.tick_params(labelsize=t_size)
ax1.set_ylabel(r'$PDF$',fontsize=f_size)
ax1.set_xticks(np.arange(0.0,1.2,0.2))
ax1.set_xlim(0,0.9)
ax1.set_ylim(0,7)
fig.tight_layout()
# plt.savefig('RMSD_AVG_Mart2.png',dpi=300,format='png')
# plt.savefig('RMSD_AVG_Mart2.svg',dpi=300,format='svg',transparent=True)


# fig.tight_layout()
# plt.savefig('RMSD_Bound_AVG_Mart2.png',dpi=300,format='png')
# plt.savefig('RMSD_Bound_AVG_Mart2.svg',dpi=300,format='svg',transparent=True)

In [ ]:
### CDF plot
pdf_3D_bd = np.histogram(merged_bd_data3D['RMSD-A'],bins=nbins,density=True)
pdf_3D_unbd = np.histogram(merged_unb_data3D['RMSD-A'],bins=nbins,density=True)

pdf_2D_bd = np.histogram(merged_bd_data2D['RMSD-A'],bins=nbins,density=True)
pdf_2D_unbd = np.histogram(merged_unb_data2D['RMSD-A'],bins=nbins,density=True)

# pdf_pseudo_bd = np.histogram(rmsd_bd_data_pseudo['RMSD-A'],bins=nbins,density=True)
# pdf_pseudo_unbd = np.histogram(rmsd_unb_data_pseudo['RMSD-A'],bins=nbins,density=True)

cdf_3D_bd = np.cumsum(pdf_3D_bd[0])*(pdf_3D_bd[1][1]-pdf_3D_bd[1][0])
cdf_3D_unbd = np.cumsum(pdf_3D_unbd[0])*(pdf_3D_unbd[1][1]-pdf_3D_unbd[1][0])

cdf_2D_bd = np.cumsum(pdf_2D_bd[0])*(pdf_2D_bd[1][1]-pdf_2D_bd[1][0])
cdf_2D_unbd = np.cumsum(pdf_2D_unbd[0])*(pdf_2D_unbd[1][1]-pdf_2D_unbd[1][0])

# cdf_pseudo_bd = np.cumsum(pdf_pseudo_bd[0])*(pdf_pseudo_bd[1][1]-pdf_pseudo_bd[1][0])
# cdf_pseudo_unbd = np.cumsum(pdf_pseudo_unbd[0])*(pdf_pseudo_unbd[1][1]-pdf_pseudo_unbd[1][0])

fig,ax = plt.subplots(figsize=(4,3))


ax.plot(pdf_3D_bd[1][:-1],cdf_3D_bd,color='navy',lw=lw+2,label='3D')
ax.plot(pdf_2D_bd[1][:-1],cdf_2D_bd,color='firebrick',lw=lw+2,label='2D')
# ax.plot(pdf_pseudo_bd[1][:-1],cdf_pseudo_bd,color='teal',lw=lw+2,label='2D-Pseudo')

# ax.legend(fontsize=legsize)
ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size+1)
ax.tick_params(labelsize=t_size+1)
ax.set_ylabel(r'$CDF$',fontsize=f_size+1)
ax.set_xticks([0.0,0.7,1.4])

fig.tight_layout()
# plt.savefig("RMSD_CDF_BoundAvgMart2.svg",dpi=600,format="svg",transparent=True)
plt.show()



"""
CDF-Unbound
"""
fig,ax = plt.subplots(figsize=(4,3))


ax.plot(pdf_3D_unbd[1][:-1],cdf_3D_unbd,color='navy',lw=lw+2,label='3D')
ax.plot(pdf_2D_unbd[1][:-1],cdf_2D_unbd,color='firebrick',lw=lw+2,label='2D')
# ax.plot(pdf_pseudo_unbd[1][:-1],cdf_pseudo_unbd,color='teal',lw=lw+2,label='2D-Pseudo')

# ax.legend(fontsize=legsize)
ax.set_xlabel(r'$RMSD (nm)$',fontsize=f_size+1)
ax.tick_params(labelsize=t_size+1)
ax.set_ylabel(r'$CDF$',fontsize=f_size+1)
ax.set_xticks([0.0,0.7,1.4])

fig.tight_layout()
# plt.savefig("RMSD_CDF_UnboundAvgMart2.svg",dpi=600,format="svg",transparent=True)
plt.show()

# RMSF w.r.t Crystal Structure


In [ ]:
## Read the 3D data
# rmsf_3D_bd_chainA = pandas.read_csv("../3D_Martini2/RMSF/CRYSTAL/RMSF_combined_Bound_CRYSTAL.dat",comment="#", sep="\t",dtype=float)
rmsf_3D_bd_chainA = pandas.read_csv("../3D_Martini2/RMSF/Short_Residuals_chainA_CRYSTAL.dat",comment="#", sep="\t",dtype=float)

rmsf_3D_unb_chainA = pandas.read_csv("../3D_Martini3/RMSF/CRYSTAL/RMSF_combined_Unbound_CRYSTAL.dat",comment="#", sep="\t",dtype=float)

mean_rmsf_3D_bd_chainA = rmsf_3D_bd_chainA.loc[:,"RMSF-0":].mean(axis=1)
sem_rmsf_3D_bd_chainA = rmsf_3D_bd_chainA.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

# mean_rmsf_3D_bd_chainB = rmsf_3D_bd_chainB.loc[:,"RMSF-0":].mean(axis=1)
# sem_rmsf_3D_bd_chainB = rmsf_3D_bd_chainB.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

mean_rmsf_3D_unb_chainA = rmsf_3D_unb_chainA.loc[:,"RMSF-0":].mean(axis=1)
sem_rmsf_3D_unb_chainA = rmsf_3D_unb_chainA.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

# mean_rmsf_3D_unb_chainB = rmsf_3D_unb_chainB.loc[:,"RMSF-0":].mean(axis=1)
# sem_rmsf_3D_unb_chainB = rmsf_3D_unb_chainB.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

print(rmsf_3D_bd_chainA)


In [ ]:
## Read 2D data

rmsf_2D_bd_chainA = pandas.read_csv("../2D_Martini2/RMSF/CRYSTAL/RMSF_combined_Bound_CRYSTAL.dat",comment="#", sep="\t",dtype=float)
# rmsf_2D_bd_chainA = pandas.read_csv("../2D_Martini2/RMSF/Short_Residuals_chainA_CRYSTAL.dat",comment="#", sep="\t",dtype=float)

# rmsf_2D_bd_chainB = pandas.read_csv("../2D_Martini3/RMSF/CRYSTAL_rmsf/RMSF_combined_Bound_CRYSTAL_chainB.dat",comment="#", sep="\t",dtype=float)

rmsf_2D_unb_chainA = pandas.read_csv("../2D_Martini2/RMSF/CRYSTAL/RMSF_combined_Unbound_CRYSTAL.dat",comment="#", sep="\t",dtype=float)
# rmsf_2D_unb_chainB = pandas.read_csv("../2D_Martini3/RMSF/CRYSTAL_rmsf/RMSF_combined_Unbound_CRYSTAL_chainB.dat",comment="#", sep="\t",dtype=float)


mean_rmsf_2D_bd_chainA = rmsf_2D_bd_chainA.loc[:,"RMSF-0":].mean(axis=1)
sem_rmsf_2D_bd_chainA = rmsf_2D_bd_chainA.loc[:,"RMSF-0":].sem(axis=1,ddof=0)     #ddof =0 means Normalizing by sqrt(block_size)

# mean_rmsf_2D_bd_chainB = rmsf_2D_bd_chainB.loc[:,"RMSF-0":].mean(axis=1)
# sem_rmsf_2D_bd_chainB = rmsf_2D_bd_chainB.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

mean_rmsf_2D_unb_chainA = rmsf_2D_unb_chainA.loc[:,"RMSF-0":].mean(axis=1)
sem_rmsf_2D_unb_chainA = rmsf_2D_unb_chainA.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

# mean_rmsf_2D_unb_chainB = rmsf_2D_unb_chainB.loc[:,"RMSF-0":].mean(axis=1)
# sem_rmsf_2D_unb_chainB = rmsf_2D_unb_chainB.loc[:,"RMSF-0":].sem(axis=1,ddof=0)

In [ ]:
rmsf_pseudo_bd_chainA = pandas.read_csv("../PseudoMart2/AllBound_Residuals_chainA_CRYSTAL.dat",comment="#", sep="\t",dtype=float)

In [ ]:
#Plotting for Bound State

fig,[ax,ax1] = plt.subplots(2,1,figsize=(6,7),sharex=True)

f_size=22
t_size=20
leg_size=14
lw=1.6

ax.plot(rmsf_3D_bd_chainA.loc[:,"Residue"],mean_rmsf_3D_bd_chainA, color="navy",label="3D",linewidth=2.0)
ax.fill_between(rmsf_3D_bd_chainA.loc[:,"Residue"], mean_rmsf_3D_bd_chainA-sem_rmsf_3D_bd_chainA, mean_rmsf_3D_bd_chainA+sem_rmsf_3D_bd_chainA, color="navy", alpha=0.2)

ax.plot(rmsf_2D_bd_chainA.loc[:,"Residue"],mean_rmsf_2D_bd_chainA, color="firebrick",label="2D",linewidth=2.0)
ax.fill_between(rmsf_2D_bd_chainA.loc[:,"Residue"], mean_rmsf_2D_bd_chainA-sem_rmsf_2D_bd_chainA, mean_rmsf_2D_bd_chainA+sem_rmsf_2D_bd_chainA, color="firebrick", alpha=0.2)

ax.plot(rmsf_pseudo_bd_chainA.loc[:,"Residue"],rmsf_pseudo_bd_chainA.loc[:,"RMSF":].mean(axis=1), color="teal",label="2D-Ps",linewidth=2.0)

ax1.plot(rmsf_3D_unb_chainA.loc[:,"Residue"],mean_rmsf_3D_unb_chainA, color="navy",label="3D",linewidth=lw)
ax1.fill_between(rmsf_3D_unb_chainA.loc[:,"Residue"], mean_rmsf_3D_unb_chainA-sem_rmsf_3D_unb_chainA, mean_rmsf_3D_unb_chainA+sem_rmsf_3D_unb_chainA, color="navy", alpha=0.2)

ax1.plot(rmsf_2D_unb_chainA.loc[:,"Residue"],mean_rmsf_2D_unb_chainA, color="firebrick",label="2D",linewidth=lw)
ax1.fill_between(rmsf_2D_unb_chainA.loc[:,"Residue"], mean_rmsf_2D_unb_chainA-sem_rmsf_2D_unb_chainA, mean_rmsf_2D_unb_chainA+sem_rmsf_2D_unb_chainA, color="firebrick", alpha=0.2)


# ax.plot(rmsf_pseudo_bd_chainA.loc[:,"Residue"],mean_rmsf_pseudo_bd_chainA, color="teal",label="Pseudo",linewidth=2.0)
# ax.fill_between(rmsf_pseudo_bd_chainA.loc[:,"Residue"], mean_rmsf_pseudo_bd_chainA-sem_rmsf_pseudo_bd_chainA, mean_rmsf_pseudo_bd_chainA+sem_rmsf_pseudo_bd_chainA, color="teal", alpha=0.2)

memb_residues = [5,15,19,75,79,82]

# ax.vlines(memb_residues,ymin=0,ymax=1.0,linestyle='dashdot',color='black')

ax1.set_xlabel("Residue",fontsize=f_size)
ax.set_ylabel(r'$RMSF |_{cry}$ (nm)',fontsize=f_size)
ax1.set_ylabel(r'$RMSF |_{cry}$ (nm)',fontsize=f_size)
ax.tick_params(axis='both', which='major', labelsize=t_size)
ax1.tick_params(axis='both', which='major', labelsize=t_size)
ax.legend(fontsize=leg_size, frameon=False)
fig.tight_layout()
plt.savefig('Bound_RMSF_Mart2.svg',format='svg',dpi=600,transparent=True)

In [ ]:
## Plot the Unbound State

fig,ax = plt.subplots(figsize=(6,4))

ax.plot(rmsf_3D_unb_chainA.loc[:,"Residue"],mean_rmsf_3D_unb_chainA, color="navy",label="3D",linewidth=lw)
ax.fill_between(rmsf_3D_unb_chainA.loc[:,"Residue"], mean_rmsf_3D_unb_chainA-sem_rmsf_3D_unb_chainA, mean_rmsf_3D_unb_chainA+sem_rmsf_3D_unb_chainA, color="navy", alpha=0.2)

ax.plot(rmsf_2D_unb_chainA.loc[:,"Residue"],mean_rmsf_2D_unb_chainA, color="firebrick",label="2D",linewidth=lw)
ax.fill_between(rmsf_2D_unb_chainA.loc[:,"Residue"], mean_rmsf_2D_unb_chainA-sem_rmsf_2D_unb_chainA, mean_rmsf_2D_unb_chainA+sem_rmsf_2D_unb_chainA, color="firebrick", alpha=0.2)

# ax.plot(rmsf_pseudo_unb_chainA.loc[:,"Residue"],mean_rmsf_pseudo_unb_chainA, color="teal",label="Pseudo",linewidth=lw)
# ax.fill_between(rmsf_pseudo_unb_chainA.loc[:,"Residue"], mean_rmsf_pseudo_unb_chainA-sem_rmsf_pseudo_unb_chainA, mean_rmsf_pseudo_unb_chainA+sem_rmsf_pseudo_unb_chainA, color="teal", alpha=0.2)

ax.set_xlabel("Residue",fontsize=f_size)
ax.set_ylabel("RMSF (nm)",fontsize=f_size)
ax.tick_params(axis='both', which='major', labelsize=t_size)
# ax.legend(fontsize=leg_size)
fig.tight_layout()
# plt.savefig('Unbound_RMSF_Mart2.svg',format='svg',dpi=600,transparent=True)